In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [ ]:
!brew uninstall chromedriver
!brew install --cask chromedriver@144

In [22]:
# Target conferences
TARGET_CONFERENCES = ['ACC', 'SEC', 'B10', 'B12', 'P12', 'BE']

# Date range
START_DATE = datetime(2024, 11, 4)
END_DATE = datetime(2025, 3, 16)

def setup_driver():
    """Setup Chrome driver with options"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in background
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver
def get_date_range():
    """Generate list of dates to scrape"""
    dates = []
    current = START_DATE
    while current <= END_DATE:
        dates.append(current)
        current += timedelta(days=1)
    return dates

def format_date_for_url(date):
    """Format date as YYYYMMDD for URL"""
    return date.strftime('%Y%m%d')

def scrape_day(driver, date):
    """Scrape data for a single day"""
    date_str = format_date_for_url(date)
    url = f"https://barttorvik.com/?year=2026&sort=&hteam=&t2value=&conlimit=All&state=All&begin={date_str}&end={date_str}&top=0&revquad=0&quad=5&venue=All&type=All&mingames=0"
    
    print(f"Scraping {date.strftime('%Y-%m-%d')}...")
    
    try:
        driver.get(url)
        
        # Wait for the table to load
        wait = WebDriverWait(driver, 20)
        
        # Wait for table to be present
        try:
            table = wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        except TimeoutException:
            print(f"  Timeout waiting for table on {date.strftime('%Y-%m-%d')}")
            return None
        
        # Give it a moment for JavaScript to finish rendering
        time.sleep(2)
        
        # Get the table HTML
        table_html = driver.find_element(By.TAG_NAME, "table").get_attribute('outerHTML')
        
        # Parse with pandas - skip the first row (D-I averages) and handle multi-level headers
        dfs = pd.read_html(table_html, header=1)  # Use row 1 as header, skip row 0
        
        if not dfs:
            print(f"  No table found for {date.strftime('%Y-%m-%d')}")
            return None
        
        df = dfs[0]  # Get first table
        
        # If there are still duplicate columns, keep only the first occurrence of each
        if df.columns.duplicated().any():
            df = df.loc[:, ~df.columns.duplicated(keep='first')]
        
        # Filter for target conferences
        conf_col = None
        for col in df.columns:
            if 'conf' in str(col).lower() or 'league' in str(col).lower():
                conf_col = col
                break
        
        if conf_col:
            # Filter by conference
            df = df[df[conf_col].isin(TARGET_CONFERENCES)]
        else:
            # If no specific conference column, filter by checking all columns
            mask = df.astype(str).apply(lambda row: any(conf in ' '.join(row.values) for conf in TARGET_CONFERENCES), axis=1)
            df = df[mask]
        
        if len(df) == 0:
            print(f"  No matching conference data found for {date.strftime('%Y-%m-%d')}")
            return None
        
        # Add scraped date column
        df['Date'] = date.strftime('%Y-%m-%d')
        
        print(f"  Found {len(df)} rows with {len(df.columns)} columns")
        return df
        
    except Exception as e:
        print(f"  Error scraping {date.strftime('%Y-%m-%d')}: {e}")
        import traceback
        traceback.print_exc()
        return None

In [23]:
def main():
    """Main scraping function"""
    print("Starting barttorvik.com scraper (Selenium)")
    print(f"Date range: {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}")
    print(f"Target conferences: {', '.join(TARGET_CONFERENCES)}")
    print("-" * 60)
    
    dates = get_date_range()
    print(f"Total days to scrape: {len(dates)}")
    print("-" * 60)
    
    # Setup driver
    print("Initializing Chrome driver...")
    driver = setup_driver()
    
    all_data = []
    
    try:
        for i, date in enumerate(dates, 1):
            df = scrape_day(driver, date)
            if df is not None and len(df) > 0:
                all_data.append(df)
            
            # Be polite - add delay between requests
            if i < len(dates):
                time.sleep(2)
        
        print("-" * 60)
        
        if all_data:
            # Combine all data - this just stacks rows
            combined_df = pd.concat(all_data, ignore_index=True)
            
            print(f"\nCombined dataframe shape: {combined_df.shape}")
            print(f"Columns: {len(combined_df.columns)}")
            
            # Save to CSV
            output_file = 'barttorvik_data.csv'
            combined_df.to_csv(output_file, index=False)
            
            print(f"\nSuccess! Scraped {len(combined_df)} total rows")
            print(f"Data saved to: {output_file}")
            print(f"\nFirst few rows:")
            print(combined_df.head())
            
            # Show unique dates scraped
            print(f"\nUnique dates scraped: {combined_df['Date'].nunique()}")
        else:
            print("\nNo data was scraped successfully")
    
    finally:
        # Always close the driver
        driver.quit()
        print("\nDriver closed")

if __name__ == "__main__":
    main()

Starting barttorvik.com scraper (Selenium)
Date range: 2024-11-04 to 2025-03-16
Target conferences: ACC, SEC, B10, B12, P12, BE
------------------------------------------------------------
Total days to scrape: 133
------------------------------------------------------------
Initializing Chrome driver...
Scraping 2024-11-04...


/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_3879/3721313143.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = date.strftime('%Y-%m-%d')


  Found 62 rows with 25 columns
Scraping 2024-11-05...


/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_3879/3721313143.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = date.strftime('%Y-%m-%d')


  Found 5 rows with 25 columns
Scraping 2024-11-06...


...

/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_3879/3721313143.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = date.strftime('%Y-%m-%d')


  Found 28 rows with 25 columns
Scraping 2025-03-15...


/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_3879/3721313143.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = date.strftime('%Y-%m-%d')


  Found 14 rows with 25 columns
Scraping 2025-03-16...
  Found 4 rows with 25 columns
------------------------------------------------------------

Combined dataframe shape: (2584, 25)
Columns: 25

Success! Scraped 2584 total rows
Data saved to: barttorvik_data.csv

First few rows:
  Rk                             Team Conf  G      Rec     AdjOE    AdjDE  \
0  1          Houston  1 seed, Finals  B12  1  1–0 0–0   134.9 7   69.4 1   
1  3                     Northwestern  B10  1  1–0 0–0  124.2 19   75.2 5   
2  4  Michigan  5 seed, Sweet Sixteen  B10  1  1–0 0–0   129.8 9  80.3 12   
3  5     Alabama  2 seed, Elite Eight  SEC  1  1–0 0–0  129.6 10  83.4 17   
4  6                    West Virginia  B12  1  1–0 0–0  125.3 16  81.6 13   

   Barthag     EFG%    EFGD%  ...      FTRD      2P%     2P%D      3P%  \
0  .9995 1  67.2 10   31.9 8  ...  31.9 114  57.4 77  37.5 23   60.0 1   
1  .9969 3  61.4 39  34.0 12  ...  34.0 128  58.7 66  41.7 51  45.0 30   
2  .9960 4   77.2 1   31.4 3  ..

/var/folders/rp/gf132r1n5bj14p__kxdf2hrm0000gp/T/ipykernel_3879/3721313143.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = date.strftime('%Y-%m-%d')


In [24]:
torvik = pd.read_csv("barttorvik_data.csv", header = 0)
torvik.columns

torvik['Team'] = torvik['Team'].str.split('  ').str[0].str.strip()
torvik.head(n = 50)

,Rk,Team,Conf,G,Rec,AdjOE,AdjDE,Barthag,EFG%,EFGD%,...,FTRD,2P%,2P%D,3P%,3P%D,3PR,3PRD,Adj T.,WAB,Date
0,1,Houston,B12,1,1–0 0–0,134.9 7,69.4 1,.9995 1,67.2 10,31.9 8,...,31.9 114,57.4 77,37.5 23,60.0 1,17.4 16,29.9 199,48.9 193,66.5 205,+0.04 59,2024-11-04
1,3,Northwestern,B10,1,1–0 0–0,124.2 19,75.2 5,.9969 3,61.4 39,34.0 12,...,34.0 128,58.7 66,41.7 51,45.0 30,11.8 5,30.3 195,32.1 49,67.7 184,+0.03 83,2024-11-04
2,4,Michigan,B10,1,1–0 0–0,129.8 9,80.3 12,.9960 4,77.2 1,31.4 3,...,15.7 15,74.4 6,37.3 22,55.6 3,10.5 4,31.6 188,27.1 22,76.8 24,+0.09 28,2024-11-04
3,5,Alabama,SEC,1,1–0 0–0,129.6 10,83.4 17,.9938 5,70.0 6,31.5 5,...,19.2 35,71.4 12,29.8 4,44.4 36,23.1 41,30.0 197,35.6 85,73.5 60,+0.05 49,2024-11-04
4,6,West Virginia,B12,1,1–0 0–0,125.3 16,81.6 13,.9929 6,66.7 11,36.8 19,...,17.6 23,73.1 10,36.4 17,41.2 54,25.0 52,56.7 17,35.3 80,70.5 131,+0.11 21,2024-11-04
5,7,Louisville,ACC,1,1–0 0–0,111.6 78,73.1 3,.9923 7,52.8 95,26.0 1,...,51.9 196,60.0 58,29.0 2,30.6 130,14.3 10,50.7 37,40.4 122,78.3 11,+0.02 93,2024-11-04
6,8,TCU,B12,1,1–0 0–0,135.0 6,92.2 45,.9877 8,64.0 26,40.7 42,...,38.9 150,60.0 58,50.0 101,46.7 20,23.5 42,40.0 112,63.0 226,69.3 157,+0.02 94,2024-11-04
7,9,Kentucky,SEC,1,1–0 0–0,125.2 17,86.2 22,.9865 9,68.5 8,39.0 33,...,17.6 23,68.3 18,46.3 81,45.8 25,18.5 24,36.9 136,39.7 112,74.2 52,+0.06 44,2024-11-04
8,10,Pittsburgh,ACC,1,1–0 0–0,123.2 23,85.3 20,.9856 10,61.5 37,35.3 14,...,34.5 130,66.7 24,33.3 8,34.8 91,26.3 67,35.4 149,32.8 53,75.5 42,+0.09 26,2024-11-04
9,12,USC,B10,1,1–0 0–0,105.7 104,77.3 8,.9735 12,56.3 64,33.3 10,...,35.2 133,61.5 52,45.0 73,32.0 111,17.6 18,39.1 123,63.0 226,66.9 196,+0.13 17,2024-11-04


In [25]:
torvik.to_csv("barttorvik_data_25.csv")